In [147]:
import numpy as np
import pandas as pd
# from shapely.geometry import Point, shape, LineString, mapping, MultiPoint
import seaborn
import geopandas as gpd
import pysal
import mapclassify
import matplotlib.pyplot as plt
from pysal.explore import esda
from pysal.lib import weights
import time
import random
from shapely import wkt
from esda.moran import Moran
from libpysal.weights import Queen, KNN


In [148]:
df_final = pd.read_csv(r"/Users/yuhan/Desktop/paper——revision/df_final.csv")
HMA_PMD_pth = r"/Users/yuhan/Desktop/paper——revision/HMA_PMD.csv"

#r"/Users/yuhan/Desktop/paper——revision/SheldusData_add.csv"
PDMData = pd.read_csv( HMA_PMD_pth )

print( df_final.columns)
print( "programFy unique ", PDMData.programFy.unique() )


Index(['Unnamed: 0', 'fips', 'Year', 'HMAprojAmot', 'NumProperty', 'NumHMA',
       'Hazard2', 'disasternumber_x', 'CropDmg', 'CropDmg21',
       'CropDmgPerCapita21', 'PropertyDmg', 'ProDmg21', 'ProDmgPerCapita21',
       'Injuries', 'InjuriesPerCapita', 'Fatalities', 'FatalitiesPerCapita',
       'Duration_Days', 'Count_h', 'ApprovLoan_h', 'Count_b', 'ApprovLoan_b',
       'HAODamage', 'HAOCount', 'HAOAproved', 'HAORepaRepl', 'HAORental',
       'HAOOther', 'HARCount', 'HARAproved', 'HARRepaRepl', 'HARRental',
       'HAROther', 'PAprojectAmt', 'PAfedshare', 'PAtotobl', 'NRI_ID', 'STATE',
       'STATEABBRV', 'COUNTY', 'COUNTYTYPE', 'STCOFIPS', 'POPULATION',
       'BUILDVALUE', 'AGRIVALUE', 'AREA', 'RISK_VALUE', 'RISK_SCORE',
       'RISK_RATNG', 'RISK_SPCTL', 'EAL_VALT', 'EAL_VALB', 'EAL_VALP',
       'EAL_VALPE', 'EAL_VALA', 'ALR_VALB', 'ALR_VALP', 'ALR_VALA',
       'ALR_NPCTL', 'SOVI_SCORE', 'SOVI_RATNG', 'SOVI_SPCTL', 'RESL_SCORE',
       'RESL_RATNG', 'RESL_SPCTL', 'RESL_VALUE

In [149]:
#print( HMAData["County FIPS"] )
PDMData['FIPS'] = PDMData['FIPS'].apply(str)

print("before ", PDMData.shape)
PDMData         = PDMData.loc[ PDMData['programFy'] > 2000 ]
print("after", PDMData.shape)

PDMData['PDMcount'] = np.where( PDMData['projectAmount'] > 0, 1, 0)

PDMData_multiple    = PDMData.groupby(['FIPS', 'state', 'county', 'programFy']).agg({
                                        'projectAmount'           :  ['sum'], 
                                        'PDMcount'                :  ['sum'], 
                                        'numberOfFinalProperties' :  ['sum']   }).reset_index()

PDMData_multiple.columns = PDMData_multiple.columns.droplevel(1)
PDMData_multiple.columns = ['fips', 'state', 'county', 'Year', 'PDMprojAmot', 'PDMcount', 'PDMNumfinalProp']

print(PDMData_multiple.head())


before  (3944, 44)
after (3800, 44)
    fips     state      county    Year  PDMprojAmot  PDMcount  PDMNumfinalProp
0   1000   Alabama   Statewide  2003.0    331167.00         1              0.0
1  10000  Delaware   Statewide  2003.0    331166.00         1              0.0
2  10001  Delaware        Kent  2018.0     50000.00         1              0.0
3  10003  Delaware  New Castle  2007.0   1841825.00         1              2.0
4  10003  Delaware  New Castle  2009.0    858549.14         1              0.0


In [150]:
df_final['fips'].replace('', np.nan, inplace=True)
df_final.dropna(subset=['fips'], inplace=True)
df_final['fips']            = df_final['fips'].astype(int)
PDMData_multiple['fips']    = PDMData_multiple['fips'].apply(str).astype(int)
print( df_final.STCOFIPS.nunique() )

data_filterall_pdm          = pd.merge( df_final, PDMData_multiple, 
                              left_on=['fips', 'Year'], right_on=['fips', 'Year'], how='left')
print( "fips ", data_filterall_pdm.STCOFIPS.nunique() )

print( data_filterall_pdm.shape )

3226
fips  3226
(92622, 74)


In [151]:
data_filterall_pdm.Year.unique()
print(data_filterall_pdm.columns)

Index(['Unnamed: 0', 'fips', 'Year', 'HMAprojAmot', 'NumProperty', 'NumHMA',
       'Hazard2', 'disasternumber_x', 'CropDmg', 'CropDmg21',
       'CropDmgPerCapita21', 'PropertyDmg', 'ProDmg21', 'ProDmgPerCapita21',
       'Injuries', 'InjuriesPerCapita', 'Fatalities', 'FatalitiesPerCapita',
       'Duration_Days', 'Count_h', 'ApprovLoan_h', 'Count_b', 'ApprovLoan_b',
       'HAODamage', 'HAOCount', 'HAOAproved', 'HAORepaRepl', 'HAORental',
       'HAOOther', 'HARCount', 'HARAproved', 'HARRepaRepl', 'HARRental',
       'HAROther', 'PAprojectAmt', 'PAfedshare', 'PAtotobl', 'NRI_ID', 'STATE',
       'STATEABBRV', 'COUNTY', 'COUNTYTYPE', 'STCOFIPS', 'POPULATION',
       'BUILDVALUE', 'AGRIVALUE', 'AREA', 'RISK_VALUE', 'RISK_SCORE',
       'RISK_RATNG', 'RISK_SPCTL', 'EAL_VALT', 'EAL_VALB', 'EAL_VALP',
       'EAL_VALPE', 'EAL_VALA', 'ALR_VALB', 'ALR_VALP', 'ALR_VALA',
       'ALR_NPCTL', 'SOVI_SCORE', 'SOVI_RATNG', 'SOVI_SPCTL', 'RESL_SCORE',
       'RESL_RATNG', 'RESL_SPCTL', 'RESL_VALUE

In [152]:
data_filterall_pdm['HazardCount'] = 1
data_filterall_pdm['HMAcount'] = np.where( data_filterall_pdm['HMAprojAmot'] > 0, 1, 0)
data_filterall_pdm['HAtotal']  = data_filterall_pdm['HAOAproved'] + data_filterall_pdm['HARAproved']

data_filterall_pdm['HAcount']  = np.where( ( ( data_filterall_pdm['HAOCount'] > 0 ) | (data_filterall_pdm['HARCount'] > 0) ), 1, 0)
data_filterall_pdm['PAcount']  = np.where(  data_filterall_pdm['PAprojectAmt'] > 0, 1, 0)

data_filterall_pdm['SBAcount']      = data_filterall_pdm['Count_h'] + data_filterall_pdm['Count_b']
data_filterall_pdm['SBAapproved']   = data_filterall_pdm['ApprovLoan_h'] + data_filterall_pdm['ApprovLoan_b']
data_filterall_pdm['SBAProjcount']  = np.where( ( (data_filterall_pdm['Count_h'] > 0) | (data_filterall_pdm['Count_b'] > 0)), 1, 0)

data_filterall_pdm[['PAprojectAmt', 'PAprojectAmt', 'PDMprojAmot', 'HAtotal', 'SBAapproved']].head()


,PAprojectAmt,PAprojectAmt,PDMprojAmot,HAtotal,SBAapproved
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN


In [153]:
data_filterall_pdm['HMAprojAmot']   = data_filterall_pdm['HMAprojAmot'].fillna(0)
data_filterall_pdm['PAprojectAmt']  = data_filterall_pdm['PAprojectAmt'].fillna(0)
data_filterall_pdm['PDMprojAmot']   = data_filterall_pdm['PDMprojAmot'].fillna(0) 
data_filterall_pdm['HAtotal']       = data_filterall_pdm['HAtotal'].fillna(0)
data_filterall_pdm['SBAapproved']   = data_filterall_pdm['SBAapproved'].fillna(0)

In [154]:
data_filterall_pdm['totalAssist'] = data_filterall_pdm['HMAprojAmot']  + data_filterall_pdm['PAprojectAmt'] + data_filterall_pdm['PDMprojAmot'] + data_filterall_pdm['HAtotal'] + data_filterall_pdm['SBAapproved'] 
data_filterall_pdm['totalDamage'] = data_filterall_pdm['CropDmg']     + data_filterall_pdm['PropertyDmg'] 


In [155]:
data_filterall_pdm.head()
data_filterall_pdm.columns
data_filterall_pdm.head()

,Unnamed: 0,fips,Year,HMAprojAmot,NumProperty,NumHMA,Hazard2,disasternumber_x,CropDmg,CropDmg21,...,HazardCount,HMAcount,HAtotal,HAcount,PAcount,SBAcount,SBAapproved,SBAProjcount,totalAssist,totalDamage
0,0,10001,2002.0,0.0,0.0,0.0,Climatological,0.0,0.0,0.0,...,1,0,0.0,0,0,NaN,0.0,0,0.0,100000.0
1,1,10001,2003.0,0.0,0.0,0.0,Climatological,0.0,0.0,0.0,...,1,0,0.0,0,0,NaN,0.0,1,0.0,1300000.0
2,2,10001,2003.0,0.0,0.0,0.0,Meteorological,0.0,0.0,0.0,...,1,0,0.0,0,0,NaN,0.0,1,0.0,8623850.0
3,3,10001,2004.0,0.0,0.0,0.0,Hydrological,0.0,0.0,0.0,...,1,0,0.0,0,0,NaN,0.0,0,0.0,625000.0
4,4,10001,2004.0,0.0,0.0,0.0,Meteorological,0.0,0.0,0.0,...,1,0,0.0,0,0,NaN,0.0,0,0.0,43000.0


In [156]:
data_fips_year = data_filterall_pdm.groupby([ 'fips', 'Year' ], as_index=False).agg({  
                                    'totalAssist'   :  ['sum'],
                                    'totalDamage'   :  ['sum'],
                                    'HMAprojAmot'   :  ['sum'],  
                                    'PAprojectAmt'  :  ['sum'], 
                                    'PDMprojAmot'   :  ['sum'],   
                                    'HAtotal'       :  ['sum'], 
                                    'SBAapproved'   :  ['sum'], 
                                    'HazardCount'   :  ['sum']}).reset_index()
data_fips_year.columns = data_fips_year.columns.droplevel(1)


print( data_fips_year.head() )

   index  fips    Year  totalAssist  totalDamage  HMAprojAmot  PAprojectAmt  \
0      0  1001  2001.0    160094.00     113000.0     160094.0           0.0   
1      1  1001  2002.0         0.00      85000.0          0.0           0.0   
2      2  1001  2003.0    741507.00    3686000.0     741507.0           0.0   
3      3  1001  2004.0   8841408.39    3689000.0     637242.0           0.0   
4      4  1001  2005.0    647286.00     740000.0     647286.0           0.0   

   PDMprojAmot     HAtotal  SBAapproved  HazardCount  logDamage  logAssist  \
0          0.0        0.00          0.0            2  11.635152  11.983523   
1          0.0        0.00          0.0            2  11.350418   0.000000   
2          0.0        0.00          0.0            3  15.120053  13.516441   
3          0.0  5613966.39    2590200.0            3  15.120866  15.994957   
4          0.0        0.00          0.0            3  13.514407  13.380545   

     logHMAA  logPAA  logPDMA     logHAA    logSBAA  
0 

In [157]:
data_filterall_pdm.Hazard2.unique()

array(['Climatological', 'Meteorological', 'Hydrological', 'Geophysical'],
      dtype=object)

In [158]:
data_filterall_climate = data_filterall_pdm.loc[ data_filterall_pdm[ 'Hazard2' ] ==  'Climatological' ].reset_index()
data_fips_year_climate = data_filterall_climate.groupby([ 'fips', 'Year' ], as_index=False).agg({  
                                    'totalAssist'   :  ['sum'],
                                    'totalDamage'   :  ['sum'],
                                    'HMAprojAmot'   :  ['sum'],  
                                    'PAprojectAmt'  :  ['sum'], 
                                    'PDMprojAmot'   :  ['sum'],   
                                    'HAtotal'       :  ['sum'], 
                                    'SBAapproved'   :  ['sum'], 
                                    'HazardCount'   :  ['sum'] }).reset_index()
data_fips_year_climate.columns = data_fips_year_climate.columns.droplevel(1)


print( data_fips_year_climate.shape )

(17846, 18)


In [159]:
data_filterall_meteo = data_filterall_pdm.loc[ data_filterall_pdm[ 'Hazard2' ] ==  'Meteorological' ].reset_index()
data_fips_year_meteo = data_filterall_meteo.groupby([ 'fips', 'Year' ], as_index=False).agg({  
                                    'totalAssist'   :  ['sum'],
                                    'totalDamage'   :  ['sum'],
                                    'HMAprojAmot'   :  ['sum'],  
                                    'PAprojectAmt'  :  ['sum'], 
                                    'PDMprojAmot'   :  ['sum'],   
                                    'HAtotal'       :  ['sum'], 
                                    'SBAapproved'   :  ['sum'], 
                                    'HazardCount'   :  ['sum']  }).reset_index()
data_fips_year_meteo.columns = data_fips_year_meteo.columns.droplevel(1)


print( data_fips_year_meteo.shape )

(44819, 18)


In [160]:

data_filterall_hydro = data_filterall_pdm.loc[ data_filterall_pdm[ 'Hazard2' ] ==  'Hydrological' ].reset_index()
data_fips_year_hydro = data_filterall_hydro.groupby([ 'fips', 'Year' ], as_index=False).agg({  
                                    'totalAssist'   :  ['sum'],
                                    'totalDamage'   :  ['sum'],
                                    'HMAprojAmot'   :  ['sum'],  
                                    'PAprojectAmt'  :  ['sum'], 
                                    'PDMprojAmot'   :  ['sum'],   
                                    'HAtotal'       :  ['sum'], 
                                    'SBAapproved'   :  ['sum'], 
                                    'HazardCount'   :  ['sum'] }).reset_index()
data_fips_year_hydro.columns = data_fips_year_hydro.columns.droplevel(1)


print( data_fips_year_hydro.shape )

(20407, 18)


In [161]:
data_filterall_geophysical = data_filterall_pdm.loc[ data_filterall_pdm[ 'Hazard2' ] ==  'Geophysical' ].reset_index()
data_fips_year_geophysical = data_filterall_geophysical.groupby([ 'fips', 'Year' ], as_index=False).agg({  
                                    'totalAssist'   :  ['sum'],
                                    'totalDamage'   :  ['sum'],
                                    'HMAprojAmot'   :  ['sum'],  
                                    'PAprojectAmt'  :  ['sum'], 
                                    'PDMprojAmot'   :  ['sum'],   
                                    'HAtotal'       :  ['sum'], 
                                    'SBAapproved'   :  ['sum'], 
                                    'HazardCount'   :  ['sum']  }).reset_index()
data_fips_year_geophysical.columns = data_fips_year_geophysical.columns.droplevel(1)


print( data_fips_year_geophysical.shape )

(1709, 18)


In [162]:
socialattr_pth = r"/Users/yuhan/Desktop/paper——revision/FLCounty_JEPR/ACS_County_2000_2019_JPER/National Risk Index/NRI_Shapefile_Counties/NRI_Shapefile_Counties.shp"
#socialattrreader = pd.read_csv( socialattr_pth )
socialattrreader = gpd.read_file( socialattr_pth )
################################################################################################
cbsa_fips_pth = r"/Users/yuhan/Desktop/paper——revision/ACS_County_level_UC/County_US_cbsa2.shp"
cbsafipsreader = gpd.read_file( cbsa_fips_pth )
################################################################################################
print( socialattrreader.crs )
print( cbsafipsreader.crs )
n_year = len( data_filterall_pdm.Year.unique() )

EPSG:3857
EPSG:3857


In [163]:
n_year


21

In [ ]:
# Start the process for each yearly assistance and damage data

In [164]:
socialattrreader    = socialattrreader.to_crs({'init':'epsg:4326'})
cbsafipsreader       = cbsafipsreader.to_crs({'init':'epsg:4326'})
socialattrreader2 = socialattrreader[~socialattrreader.STATEABBRV.isin([
    'HI','AK', 'AS', 'GU', 'PR', 'VI','MP'])]
cbsafipsreader2 = cbsafipsreader[ [ "STCOFIPS", "NAME" ] ]
cbsafipsreader2["name2"] = cbsafipsreader2["NAME"].str.split(",")[0]

social_df = pd.merge( socialattrreader2, cbsafipsreader2, left_on=['STCOFIPS'], right_on=['STCOFIPS'], how='left')
social_df.head()

print( socialattrreader2.shape )


/Users/yuhan/opt/anaconda3/envs/hazard_env/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/yuhan/opt/anaconda3/envs/hazard_env/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


(3108, 467)


/var/folders/t6/8gnslc_j433220_1kgd9hv_c0000gn/T/ipykernel_7380/207981358.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cbsafipsreader2["name2"] = cbsafipsreader2["NAME"].str.split(",")[0]


In [165]:
social_df['x'] = social_df.centroid.x
social_df['y'] = social_df.centroid.y

/var/folders/t6/8gnslc_j433220_1kgd9hv_c0000gn/T/ipykernel_7380/2626863007.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  social_df['x'] = social_df.centroid.x
/var/folders/t6/8gnslc_j433220_1kgd9hv_c0000gn/T/ipykernel_7380/2626863007.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  social_df['y'] = social_df.centroid.y


In [166]:
social_df['fips'] = pd.to_numeric( social_df[ 'STCOFIPS' ].apply(str) )
print( social_df.columns )

Index(['NRI_ID', 'STATE', 'STATEABBRV', 'STATEFIPS', 'COUNTY', 'COUNTYTYPE',
       'COUNTYFIPS', 'STCOFIPS', 'POPULATION', 'BUILDVALUE',
       ...
       'WNTW_RISKR', 'NRI_VER', 'Shape_Leng', 'Shape_Area', 'geometry', 'NAME',
       'name2', 'x', 'y', 'fips'],
      dtype='object', length=472)


In [169]:
social_df_nogeometry = social_df[ [ 'STCOFIPS', 'POPULATION', 
                                   'BUILDVALUE', 'SOVI_SCORE',  'fips', 'x', 'y' ] ]
social_df_nogeometry['logPOP'] = np.log( social_df_nogeometry['POPULATION'] + 1 )
social_df_nogeometry['logBUIVAL'] = np.log( social_df_nogeometry['BUILDVALUE'] + 1 )

/var/folders/t6/8gnslc_j433220_1kgd9hv_c0000gn/T/ipykernel_7380/133878786.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  social_df_nogeometry['logPOP'] = np.log( social_df_nogeometry['POPULATION'] + 1 )
/var/folders/t6/8gnslc_j433220_1kgd9hv_c0000gn/T/ipykernel_7380/133878786.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  social_df_nogeometry['logBUIVAL'] = np.log( social_df_nogeometry['BUILDVALUE'] + 1 )


In [170]:
#social_df_readOGR = social_df[ [ 'STCOFIPS', 'geometry', 'fips' ] ]
#state_ptho = r"/Users/yuhan/Desktop/paper——revision/export_results/county_boundary.shp"
#df_joini_readOGR = gpd.GeoDataFrame( social_df_readOGR , geometry = 'geometry', crs="EPSG:4326")
#df_joini_readOGR.to_file( state_ptho ,driver='ESRI Shapefile')
print( social_df_nogeometry.shape )

(3108, 9)


In [171]:
social_df_nogeometry_n          =   social_df_nogeometry.loc[ social_df_nogeometry.index.repeat( n_year ) ]
social_df_nogeometry_n['Year']  =   2001 + social_df_nogeometry_n.groupby(level=0).cumcount()

In [172]:
social_df_nogeometry_n.head()

,STCOFIPS,POPULATION,BUILDVALUE,SOVI_SCORE,fips,x,y,logPOP,logBUIVAL,Year
0,01001,58764,9.123274e+09,51.299999,1001,-86.642759,32.534932,10.981302,22.934095,2001
0,01001,58764,9.123274e+09,51.299999,1001,-86.642759,32.534932,10.981302,22.934095,2002
0,01001,58764,9.123274e+09,51.299999,1001,-86.642759,32.534932,10.981302,22.934095,2003
0,01001,58764,9.123274e+09,51.299999,1001,-86.642759,32.534932,10.981302,22.934095,2004
0,01001,58764,9.123274e+09,51.299999,1001,-86.642759,32.534932,10.981302,22.934095,2005


In [173]:
social_df_nogeometry_n.shape
social_df_nogeometry_n['Year'].max()

2021

In [176]:
data_fips_year['Year'] = data_fips_year['Year'].astype(int)
data_fips_year.head()


,index,fips,Year,totalAssist,totalDamage,HMAprojAmot,PAprojectAmt,PDMprojAmot,HAtotal,SBAapproved,HazardCount,logDamage,logAssist,logHMAA,logPAA,logPDMA,logHAA,logSBAA
0,0,1001,2001,160094.00,113000.0,160094.0,0.0,0.0,0.00,0.0,2,11.635152,11.983523,11.983523,0.0,0.0,0.000000,0.000000
1,1,1001,2002,0.00,85000.0,0.0,0.0,0.0,0.00,0.0,2,11.350418,0.000000,0.000000,0.0,0.0,0.000000,0.000000
2,2,1001,2003,741507.00,3686000.0,741507.0,0.0,0.0,0.00,0.0,3,15.120053,13.516441,13.516441,0.0,0.0,0.000000,0.000000
3,3,1001,2004,8841408.39,3689000.0,637242.0,0.0,0.0,5613966.39,2590200.0,3,15.120866,15.994957,13.364906,0.0,0.0,15.540768,14.767246
4,4,1001,2005,647286.00,740000.0,647286.0,0.0,0.0,0.00,0.0,3,13.514407,13.380545,13.380545,0.0,0.0,0.000000,0.000000


In [191]:
def classifyYear( x ):
    if x < 2006:
        return 1
    elif x < 2011:
        return 2
    elif x < 2016:
        return 3
    else:
        return 4

social_df_nogeometry_n['YearClass'] = social_df_nogeometry_n['Year'].map( classifyYear )
print( social_df_nogeometry_n.YearClass.unique() )

[1 2 3 4]


In [192]:
social_df0 = pd.merge( social_df_nogeometry_n, data_fips_year, 
                     left_on=['fips', 'Year'], right_on=['fips', 'Year'], how='left')
social_df0.head()
social_df0 = social_df0.fillna(0)
social_df0.head()


,STCOFIPS,POPULATION,BUILDVALUE,SOVI_SCORE,fips,x,y,logPOP,logBUIVAL,Year,...,HAtotal,SBAapproved,HazardCount,logDamage,logAssist,logHMAA,logPAA,logPDMA,logHAA,logSBAA
0,01001,58764,9.123274e+09,51.299999,1001,-86.642759,32.534932,10.981302,22.934095,2001,...,0.00,0.0,2.0,11.635152,11.983523,11.983523,0.0,0.0,0.000000,0.000000
1,01001,58764,9.123274e+09,51.299999,1001,-86.642759,32.534932,10.981302,22.934095,2002,...,0.00,0.0,2.0,11.350418,0.000000,0.000000,0.0,0.0,0.000000,0.000000
2,01001,58764,9.123274e+09,51.299999,1001,-86.642759,32.534932,10.981302,22.934095,2003,...,0.00,0.0,3.0,15.120053,13.516441,13.516441,0.0,0.0,0.000000,0.000000
3,01001,58764,9.123274e+09,51.299999,1001,-86.642759,32.534932,10.981302,22.934095,2004,...,5613966.39,2590200.0,3.0,15.120866,15.994957,13.364906,0.0,0.0,15.540768,14.767246
4,01001,58764,9.123274e+09,51.299999,1001,-86.642759,32.534932,10.981302,22.934095,2005,...,0.00,0.0,3.0,13.514407,13.380545,13.380545,0.0,0.0,0.000000,0.000000


In [201]:


social_df0[ ['STCOFIPS', 'fips', 'POPULATION', 'BUILDVALUE', 'Year',
                           'SOVI_SCORE', 'HazardCount', 'x', 'y',
                           'totalDamage', 'totalAssist', 'HMAprojAmot',
                           'PAprojectAmt', 'PDMprojAmot', 'HAtotal', 'SBAapproved', 'YearClass' ] ].head()

social_df01 = social_df0[ ['STCOFIPS', 'fips', 'POPULATION', 'BUILDVALUE', 'Year',
                           'SOVI_SCORE', 'HazardCount', 'x', 'y',
                           'totalDamage', 'totalAssist', 'HMAprojAmot',
                           'PAprojectAmt', 'PDMprojAmot', 'HAtotal', 'SBAapproved', 'YearClass' ] ]


In [228]:
social_df01_catey = social_df01.groupby([ 'fips', 'YearClass' ], as_index=False).agg({  
                                    'POPULATION'   :  ['sum'],
                                    'BUILDVALUE'   :  ['sum'],
                                    'SOVI_SCORE'   :  ['sum'],
                                    'totalAssist'   :  ['sum'],
                                    'totalDamage'   :  ['sum'],
                                    'HMAprojAmot'   :  ['sum'],  
                                    'PAprojectAmt'  :  ['sum'], 
                                    'PDMprojAmot'   :  ['sum'],   
                                    'HAtotal'       :  ['sum'], 
                                    'SBAapproved'   :  ['sum'], 
                                    'HazardCount'   :  ['sum'],
                                    'x': ['mean'],
                                    'y': ['mean']    }).reset_index()
social_df01_catey.columns   = social_df01_catey.columns.droplevel(1)


In [229]:
social_df01_catey.head()

,index,fips,YearClass,POPULATION,BUILDVALUE,SOVI_SCORE,totalAssist,totalDamage,HMAprojAmot,PAprojectAmt,PDMprojAmot,HAtotal,SBAapproved,HazardCount,x,y
0,0,1001,1,293820,4.561637e+10,256.499996,1.039030e+07,8313000.0,2186129.0,0.0,0.0,5.613966e+06,2590200.0,13.0,-86.642759,32.534932
1,1,1001,2,293820,4.561637e+10,256.499996,5.298605e+06,11847500.0,968199.0,0.0,0.0,4.038606e+06,291800.0,7.0,-86.642759,32.534932
2,2,1001,3,293820,4.561637e+10,256.499996,4.196874e+06,2763600.0,248655.0,0.0,0.0,3.796919e+06,151300.0,2.0,-86.642759,32.534932
3,3,1001,4,352584,5.473964e+10,307.799995,0.000000e+00,50000.0,0.0,0.0,0.0,0.000000e+00,0.0,2.0,-86.642759,32.534932
4,4,1003,1,1156825,2.298424e+11,155.150003,8.945502e+09,49950000.0,658668177.0,0.0,0.0,7.385878e+09,900956100.0,83.0,-87.723352,30.736470


In [230]:
social_df01_catey['logDamage']   = np.log( social_df01_catey['totalDamage'] +1 )
social_df01_catey['logAssist']   = np.log( social_df01_catey['totalAssist'] +1 )

social_df01_catey['logPOP']   = np.log( social_df01_catey['POPULATION'] +1 )
social_df01_catey['logBUILDVAL']   = np.log( social_df01_catey['BUILDVALUE'] +1 )


social_df01_catey['logHMAA']     = np.log( social_df01_catey['HMAprojAmot']  + 1 )
social_df01_catey['logPAA']      = np.log( social_df01_catey['PAprojectAmt'] + 1 )
social_df01_catey['logPDMA']     = np.log( social_df01_catey['PDMprojAmot'] + 1 )
social_df01_catey['logHAA']      = np.log( social_df01_catey['HAtotal'] + 1 )
social_df01_catey['logSBAA']     = np.log( social_df01_catey['SBAapproved'] + 1 )

social_df01_catey.columns

Index(['index', 'fips', 'YearClass', 'POPULATION', 'BUILDVALUE', 'SOVI_SCORE',
       'totalAssist', 'totalDamage', 'HMAprojAmot', 'PAprojectAmt',
       'PDMprojAmot', 'HAtotal', 'SBAapproved', 'HazardCount', 'x', 'y',
       'logDamage', 'logAssist', 'logPOP', 'logBUILDVAL', 'logHMAA', 'logPAA',
       'logPDMA', 'logHAA', 'logSBAA'],
      dtype='object')

In [231]:
social_df02_catey  = social_df01_catey[ ['fips', 'YearClass', 'SOVI_SCORE', 'x', 'y', 
                                                 'logDamage', 'logAssist',
                                                 'logPOP', 'logBUILDVAL', 'logHMAA', 
                                                 'logPAA', 'logPDMA', 'logHAA', 'logSBAA' ] ]


In [232]:

social_df02_catey.to_csv( r"/Users/yuhan/Desktop/paper——revision/export_results/social_df0.csv", index=False )


In [233]:
geometry0 = gpd.points_from_xy( x= social_df02_catey.x , y= social_df02_catey.y )
gdf0 = gpd.GeoDataFrame( social_df02_catey, geometry= geometry0 , crs="EPSG:4326" )

In [234]:
state_pth0 = r"/Users/yuhan/Desktop/paper——revision/export_results/social_df0.shp"
gdf0.to_file( state_pth0 ,driver='ESRI Shapefile')


/var/folders/t6/8gnslc_j433220_1kgd9hv_c0000gn/T/ipykernel_7380/827370526.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf0.to_file( state_pth0 ,driver='ESRI Shapefile')
